# Лабораторная №1
## Задание
Для изготовление книжной полки требуется 3 листа древесной плиты размером 80x20 см. и 2 листа размером 65x25 см.

Мастерская может закупить не более 20 листов древесной плиты размером 100x100 см. по цене 400 рублей за лист. Книжная полка продаётся по цене 1000 рублей. Дополнительные затраты на вспомогательные материалы и оплату работы составляют 200 рублей на каждую книжную полку.

**Требуется:**

Определить план работы мастерской, обеспечивающий максимальную прибыль

# Решение
Рассмотрим способы разделение листа 100х100 см. на листы 80х20 и 65х20 см.

Всего существует 6 способов разделения с точностью до незначимых сдвигов разрезов и поворотов заготовки:

| **Способ** | **80х20 (длинная)** | **65х25 (короткая)** |
|:----------:|:---------:|:---------:|
|     _1_    |     0     |     5     |
|     _2_    |     1     |     4     |
|     _3_    |     2     |     3     |
|     _4_    |     3     |     2     |
|     _5_    |     4     |     1     |
|     _6_    |     6     |     0     |